In [191]:
import simpy
import random
import numpy as np
import pandas as pd
import simpy
import seaborn as sns
from scipy.stats import norm

In [192]:
# Parameters
START_TIME = 12 * 3600  # Start time at 12 pm in seconds
# Base processing times in seconds, from 0.5 to 1.2 minutes
ORDER_PROCESSING_TIME_BASE = [x * 60 for x in np.arange(0.5, 1.3, 0.1)]
ORDER_PROCESSING_TIME_PER_SCOOP = 0.2 * 60  # in seconds
REFILL_TIME = 3 * 60 # in seconds
STORE_HOURS = (12 * 3600, 19 * 3600)  # Store hours from 1 pm to 7 pm in seconds
PEAK_TIMES = [(12 * 3600, 13 * 3600), (17 * 3600, 19 * 3600)]  # Peak times in seconds
#PEAK_ARRIVAL_RATE_MULTIPLIER = 1.2  # Arrival rate multiplier during peak hours
BUCKET_SIZE = 80
LAST_CUSTOMER_TIME = 15 * 60 # don't take order 5min until closing

# Define employee shifts in seconds from the start of the store's operation
EMPLOYEE_SHIFTS = [
    (0, 7 * 3600),
    (0, 7 * 3600),
    (0 * 3600, 2 * 3600),
    (5 * 3600, 7 * 3600)
]

FLAVOR_PROBABILITIES = {
    'Vanilla': 0.10,
    'Chocolate': 0.10,
    'Strawberry': 0.05,
    'Mango': 0.02,
    'Pistachio': 0.10,
    'Hazelnut': 0.08,
    'Lemon': 0.02,
    'Coffee': 0.05,
    'Coconut': 0.01,
    'Raspberry': 0.01,
    'Tiramisu': 0.04,
    'Amarena': 0.02,
    'Snickers': 0.08,
    'Straciatela': 0.03,
    'Mint Chocolate': 0.03,
    'Caramel': 0.05,
    'Bacio': 0.01,
    'Fior di Latte': 0.02,
    'Mars': 0.08,
    'Oreo': 0.10
}

FLAVORS = list(FLAVOR_PROBABILITIES.keys())
WEIGHTS = list(FLAVOR_PROBABILITIES.values())

**Event Logger**


In [193]:
class EventLogger:
    def __init__(self):
        self.logs = []
        self.replication = None  # Replication number
        self.seed = None  # Seed used for the replication
        self.base_processing_speed = None  # Base processing speed
        
    def set_replication_info(self, replication, seed, base_processing_speed):
        self.replication = replication
        self.seed = seed
        self.base_processing_speed = base_processing_speed

    def log_event(self, event_time, event_name, customer_id=None, queue_length=None, scoops=None, flavors=None, peak_hours=None, bucket_levels=None, working_employees=None):
        log_entry = {
            'replication_id': self.replication,  # Replication info
            'seed': self.seed,  # Seed info
            'base_processing_speed': self.base_processing_speed,  # Base processing speed
            'event_time': (event_time + START_TIME) / 3600,  # Convert adjusted time to hours
            'event_name': event_name,
            'customer_id': customer_id,
            'queue_length': queue_length,
            'scoops': scoops,
            'flavors': flavors,
            'peak_hours': peak_hours,
            'bucket_levels': bucket_levels,
            'working_employees': working_employees  # Add working_employees to the log entry
        }
        self.logs.append(log_entry)
        
    def log_queue_length(self, time, queue_length):
        self.logs.append({'replication_id': self.replication, 'seed': self.seed, #replication info
                          'base_processing_speed': self.base_processing_speed,  # Base processing speed
                          'event_time': time, 'event_name': 'queue_monitor', 'queue_length': queue_length #queue length info
        })

    def get_logs_df(self):
        return pd.DataFrame(self.logs)

    def dump_logs_df(self, filepath=None):
        if filepath is None:
            filepath = "logs.csv"
        self.get_logs_df().to_csv(filepath, index=False)

# Event logger instance
event_logger = EventLogger()

**Resources: Ice Cream Shop**
The IceCreamStand class models the ice cream shop, including employees, ice cream buckets, and customer queue.

In [194]:
class IceCreamStand:
    def __init__(self, env, employee_shifts, base_processing_time):
        self.env = env
        self.queue = []
        self.base_processing_time = base_processing_time
        self.buckets = {flavor: IceCreamBucket(env, flavor) for flavor in FLAVORS}
        self.employees = [Employee(env, self, i, shift) for i, shift in enumerate(employee_shifts)]
        self.env.process(self.log_working_employees())  # Start the logging process

    def join_queue(self, customer):
        self.queue.append(customer)
        event_logger.log_event(self.env.now, 'Join Queue', customer_id=customer.id, queue_length=len(self.queue))
        impatience_event = self.env.timeout(customer.patience)  # Event for the customer's patience timeout
        service_event = self.env.process(self.wait_for_service(customer))  # Event for waiting to be served

        result = yield impatience_event | service_event  # Wait for either the impatience or service event

        if impatience_event in result and not customer.served:
            # Customer left due to impatience
            event_logger.log_event(self.env.now, 'Left Due to Impatience', customer.id)
            if customer in self.queue:
                self.queue.remove(customer)  # Remove customer from queue
        else:
            # Customer got served
            customer.served = True
            yield service_event

    def wait_for_service(self, customer):
        while True:
            if customer in self.queue and self.queue[0] == customer:
                yield self.env.timeout(0)  # Customer's turn to be served
                break
            yield self.env.timeout(1)  # Check the queue periodically

    def process_order(self, customer, employee):
        # Determine number of scoops with weighted probabilities
        scoops = random.choices([1, 2, 3, 5], weights=[5, 3, 1, 1])[0]
        flavors = random.choices(FLAVORS, WEIGHTS, k=scoops)
        event_logger.log_event(self.env.now, 'Order', customer_id=customer.id, scoops=scoops, flavors=flavors, peak_hours=self.is_peak_time())

        # Processing time for the order
        processing_time = self.base_processing_time + ORDER_PROCESSING_TIME_PER_SCOOP * (scoops - 1)
        yield self.env.timeout(processing_time)

        # Serve scoops and log bucket levels after each scoop
        for flavor in flavors:
            bucket = self.buckets[flavor]
            if bucket.quantity == 0:
                if not bucket.refilling:
                    bucket.refilling = True
                    employee.working = False  # Mark employee as not working during refill
                    employee.refilling = True  # Mark employee as refilling
                    yield self.env.process(self.refill_bucket(employee, bucket))  # Employee stops to refill the bucket
                    employee.refilling = False  # Mark employee as not refilling
                    employee.working = True  # Mark employee as working after refill
                while bucket.refilling:
                    yield self.env.timeout(1)  # Wait until the bucket is refilled
            bucket.serve_scoop()
            event_logger.log_event(self.env.now, 'Bucket Level', bucket_levels={flavor: bucket.quantity})

        event_logger.log_event(self.env.now, 'Processed', customer_id=customer.id)
        yield self.env.timeout(1 * 60)  # Simulate payment time
        event_logger.log_event(self.env.now, 'Leave', customer_id=customer.id)
        customer.served = True  # Mark the customer as served

    def refill_bucket(self, employee, bucket):
        event_logger.log_event(self.env.now, 'Refill', bucket_levels={bucket.flavor: bucket.quantity})
        yield self.env.timeout(REFILL_TIME)
        bucket.quantity = BUCKET_SIZE
        bucket.refilling = False
        event_logger.log_event(self.env.now, 'Refilled', bucket_levels={bucket.flavor: bucket.quantity})

    def is_peak_time(self):
        current_time = self.env.now + START_TIME
        return any(start <= current_time % 86400 < end for start, end in PEAK_TIMES)

    def log_working_employees(self):
        while True:
            working_employees = sum(1 for employee in self.employees if employee.working and not employee.refilling)
            event_logger.log_event(self.env.now, 'Working Employees', working_employees=working_employees)
            yield self.env.timeout(10 * 60)  # Log every 10 minutes


**Entities: Customer, Employee, IceCreamBucket**

The Customer, Employee, and IceCreamBucket classes model the respective entities in the simulation.

In [195]:
class Customer:
    def __init__(self, env, ice_cream_stand, id):
        self.env = env
        self.ice_cream_stand = ice_cream_stand
        self.id = id
        self.arrival_time = env.now
        self.served = False
        self.patience = random.uniform(10 * 60, 20 * 60)

    def visit_stand(self):
        event_logger.log_event(self.env.now, 'Arrival', self.id)
        yield self.env.process(self.ice_cream_stand.join_queue(self))

class Employee:
    def __init__(self, env, ice_cream_stand, id, shift):
        self.env = env
        self.ice_cream_stand = ice_cream_stand
        self.id = id
        self.shift_start, self.shift_end = shift
        self.working = False  # Track if the employee is working
        self.refilling = False  # Track if the employee is refilling
        self.env.process(self.work())

    def work(self):
        while True:
            current_time = self.env.now
            if self.shift_start <= current_time < self.shift_end:
                if not self.ice_cream_stand.queue or self.refilling:
                    self.working = False  # Not working if no customers or refilling
                    yield self.env.timeout(1)
                    continue

                self.working = True  # Start working
                customer = self.ice_cream_stand.queue.pop(0)
                yield self.env.process(self.ice_cream_stand.process_order(customer, self))
                self.working = True  # Stop working
            else:
                self.working = False  # Not working if shift ended
                yield self.env.timeout(1)

class IceCreamBucket:
    def __init__(self, env, flavor, capacity=BUCKET_SIZE):
        self.env = env
        self.flavor = flavor
        self.capacity = capacity
        self.quantity = capacity
        self.refilling = False  # Add a flag to track refill status

    def serve_scoop(self):
        if self.quantity > 0:
            self.quantity -= 1
        if self.quantity < 10 and not self.refilling:
            self.refilling = True
            self.env.process(self.refill())

    def refill(self):
        event_logger.log_event(self.env.now, 'Refill', bucket_levels={self.flavor: self.quantity})
        yield self.env.timeout(REFILL_TIME)
        self.quantity = BUCKET_SIZE
        self.refilling = False  # Reset the flag after refilling
        event_logger.log_event(self.env.now, 'Refilled', bucket_levels={self.flavor: self.quantity})


**Customer Arrival Process**

The customer_arrivals function generates customers at different intervals for peak and non-peak hours.

In [196]:
# Define the Normal distribution parameters
np_mean, np_std = 61, 45
p_mean, p_std = 42, 41

MAX_CUSTOMERS = 1000

def inverse_transform_normal(mean, std):
    R = np.random.uniform(0, 1)
    return norm.ppf(R, loc=mean, scale=std)

def customer_arrivals(env, stand):
    customer_id = 0
    while customer_id < MAX_CUSTOMERS:
        current_time = env.now + START_TIME
        # Stop taking new customers n minutes before the store closes
        if current_time >= STORE_HOURS[1] - LAST_CUSTOMER_TIME:
            break

        # Determine if current time is within peak times
        if any(start <= current_time % 86400 < end for start, end in PEAK_TIMES):
            arrival_interval = inverse_transform_normal(p_mean, p_std)
        else:
            arrival_interval = inverse_transform_normal(np_mean, np_std)

        # Ensure the interval is positive
        arrival_interval = max(0, arrival_interval)

        yield env.timeout(arrival_interval)

        if current_time < STORE_HOURS[0] or current_time >= STORE_HOURS[1]:
            continue

        customer = Customer(env, stand, customer_id)
        env.process(customer.visit_stand())
        customer_id += 1


**Running simulation**


In [197]:
# Running simulation
print('Running Simulation...')
N_REPLICATIONS = 5  # Number of Replications

# Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

# Get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
# We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)
print(list_of_seeds)

# Loop over different base processing speeds
for base_processing_speed in ORDER_PROCESSING_TIME_BASE:
    print(f"Simulating with base processing speed: {base_processing_speed / 60:.1f} minutes per order")
    for i, seed in enumerate(list_of_seeds):
        print(f'  Running Replication {i} with seed: {seed} ...')

        # Set random seed
        np.random.seed(seed)

        # Set replication id, seed, and base processing speed in the event logger
        event_logger.set_replication_info(i, seed, base_processing_speed)

        # Simulation
        env = simpy.Environment()
        stand = IceCreamStand(env, EMPLOYEE_SHIFTS, base_processing_speed)
        env.process(customer_arrivals(env, stand))

        env.run(until=STORE_HOURS[1] - START_TIME)
        
        # Log the store closing time
        event_logger.log_event(env.now, 'Store closes')

print('... Done')



Running Simulation...
[41 26  9  4 11]
Simulating with base processing speed: 0.5 minutes per order
  Running Replication 0 with seed: 41 ...
  Running Replication 1 with seed: 26 ...
  Running Replication 2 with seed: 9 ...
  Running Replication 3 with seed: 4 ...
  Running Replication 4 with seed: 11 ...
Simulating with base processing speed: 0.6 minutes per order
  Running Replication 0 with seed: 41 ...
  Running Replication 1 with seed: 26 ...
  Running Replication 2 with seed: 9 ...
  Running Replication 3 with seed: 4 ...
  Running Replication 4 with seed: 11 ...
Simulating with base processing speed: 0.7 minutes per order
  Running Replication 0 with seed: 41 ...
  Running Replication 1 with seed: 26 ...
  Running Replication 2 with seed: 9 ...
  Running Replication 3 with seed: 4 ...
  Running Replication 4 with seed: 11 ...
Simulating with base processing speed: 0.8 minutes per order
  Running Replication 0 with seed: 41 ...
  Running Replication 1 with seed: 26 ...
  Running

In [198]:
# Get logs DataFrame from event logger
event_logger.dump_logs_df("Employee_Speed.csv")

logs_df = event_logger.get_logs_df()
logs_df

,replication_id,seed,base_processing_speed,event_time,event_name,customer_id,queue_length,scoops,flavors,peak_hours,bucket_levels,working_employees
0,0,41,30.0,12.000000,Working Employees,NaN,NaN,NaN,None,None,None,0.0
1,0,41,30.0,12.004018,Arrival,0.0,NaN,NaN,None,None,None,NaN
2,0,41,30.0,12.004018,Join Queue,0.0,1.0,NaN,None,None,None,NaN
3,0,41,30.0,12.004018,Arrival,1.0,NaN,NaN,None,None,None,NaN
4,0,41,30.0,12.004018,Join Queue,1.0,2.0,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
125574,4,11,72.0,18.966667,Bucket Level,NaN,NaN,NaN,None,None,{'Lemon': 56},NaN
125575,4,11,72.0,18.966667,Processed,443.0,NaN,NaN,None,None,None,NaN
125576,4,11,72.0,18.980556,Leave,442.0,NaN,NaN,None,None,None,NaN
125577,4,11,72.0,18.983333,Leave,443.0,NaN,NaN,None,None,None,NaN
